In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import convnet_builder as bare_model_builder

In [2]:
PROJECT_NAME = 'convnet'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    # Convolutional network params
    hp_n_conv_blocks = hp.Int("n_conv_blocks", min_value=1, max_value=3, step=1)
    hp_n_conv_layers = hp.Int("n_conv_layers", min_value=1, max_value=3, step=1)
    hp_filters_mult = hp.Choice("conv_filters_mult", values=[1, 2, 4, 8])
    hp_conv_spatial_dropout = hp.Choice("conv_spatial_dropout", values=[0.0, 0.1, 0.2])

    # MLP at the end params
    hp_mlp_n_hidden_layers = hp.Int("n_mlp_hidden_layers", min_value=0, max_value=3, step=1, default=0)
    hp_mlp_units_mult, hp_mlp_dropout = None, None
    if hp_mlp_n_hidden_layers > 0:
        hp_mlp_units_mult = hp.Choice("mlp_units_mult", values=[1, 2, 4, 8, 16], default=4)
        hp_mlp_dropout = hp.Choice("mlp_dropout", values=[0.0, 0.2, 0.5])

    # Other params
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)

    model = bare_model_builder(hp_n_conv_blocks, hp_n_conv_layers, hp_filters_mult, hp_conv_spatial_dropout,
                               hp_mlp_n_hidden_layers, hp_mlp_units_mult, hp_mlp_dropout, hp_batch_normalization,
                               hp_input_batch_normalization)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 conv1d (Conv1D)             (None, 24, 8)             32        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 8)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 96)                0         
                                                                 
 dense (Dense)               (None, 1)                 97        
                                                                 
Total params: 129
Trainable params: 129
Non-trainable params: 0
__________________________________________________________

In [6]:
model_callbacks = [
    callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)
]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [8]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_conv_blocks': 1, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 2, 'batch_normalization': True, 'input_batch_normalization': True, 'mlp_units_mult': 4, 'mlp_dropout': 0.2}
Number of parameters: 16369
========== Model 1 ==========
{'n_conv_blocks': 2, 'n_conv_layers': 2, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 3, 'batch_normalization': True, 'input_batch_normalization': False, 'mlp_units_mult': 8, 'mlp_dropout': 0.5}
Number of parameters: 41233
========== Model 2 ==========
{'n_conv_blocks': 2, 'n_conv_layers': 1, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 0, 'batch_normalization': False, 'input_batch_normalization': True, 'mlp_units_mult': 2, 'mlp_dropout': 0.2}
Number of parameters: 25777
========== Model 3 ==========
{'n_conv_blocks': 2, 'n_conv_layers': 3, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 0,

# Cross-validation for top 5 models

In [9]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC,
                                           overwrite=OVERWRITE)
model_scores = cross_validator()

{'n_conv_blocks': 1, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 2, 'batch_normalization': True, 'input_batch_normalization': True, 'mlp_units_mult': 4, 'mlp_dropout': 0.2}
Number of parameters: 16369
Got score: 8.6875 (8.6926, 8.6824)
Got score: 8.3864 (8.3377, 8.4351)
Got score: 8.2769 (8.3760, 8.1778)
Got score: 8.5380 (8.5629, 8.5131)
Got score: 8.7846 (8.7500, 8.8193)


{'n_conv_blocks': 2, 'n_conv_layers': 2, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 3, 'batch_normalization': True, 'input_batch_normalization': False, 'mlp_units_mult': 8, 'mlp_dropout': 0.5}
Number of parameters: 41233
Got score: 85.1805 (161.4244, 8.9366)
Got score: 66.8397 (8.5015, 125.1779)
Got score: 33.7293 (8.5296, 58.9290)
Got score: 138.5524 (195.6927, 81.4120)
Got score: 9.3482 (9.7569, 8.9395)


{'n_conv_blocks': 2, 'n_conv_layers': 1, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 0, 'batch_normalization': False, 'input_batch_normalization': True, 'mlp_units_mult': 2, 'mlp_dropout': 0.2}
Number of parameters: 25777
Got score: 8.8196 (8.8545, 8.7847)
Got score: 8.6826 (8.8821, 8.4830)
Got score: 8.2013 (8.1434, 8.2591)
Got score: 8.4961 (8.5495, 8.4427)
Got score: 8.8874 (8.9295, 8.8452)


{'n_conv_blocks': 2, 'n_conv_layers': 3, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 0, 'batch_normalization': False, 'input_batch_normalization': True, 'mlp_units_mult': 8, 'mlp_dropout': 0.0}
Number of parameters: 149041
Got score: 9.1780 (9.3384, 9.0177)
Got score: 16.1133 (8.5631, 23.6636)
Got score: 8.1411 (8.1177, 8.1644)
Got score: 8.5712 (8.5712, 8.5712)
Got score: 8.8865 (8.8273, 8.9456)


{'n_conv_blocks': 1, 'n_conv_layers': 1, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 1, 'batch_normalization': False, 'input_batch_normalization': True, 'mlp_units_mult': 2, 'mlp_dropout': 0.0}
Number of parameters: 6465
Got score: 8.8826 (8.8742, 8.8910)
Got score: 8.6055 (8.7584, 8.4527)
Got score: 8.1408 (8.1452, 8.1365)
Got score: 8.5688 (8.5976, 8.5400)
Got score: 8.8680 (8.8864, 8.8496)


In [10]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.53,0.19,16369
1,66.73,44.45,41233
2,8.62,0.25,25777
3,10.18,2.99,149041
4,8.61,0.27,6465
